In [1]:
import fsl.wrappers
import mne_nirs
import nilearn
import mne
import mne_nirs
import dipy
import xml.etree.ElementTree as ET
import os.path as op
from fsleyes.views.orthopanel import OrthoPanel
import nibabel as nib
from nilearn.datasets import fetch_development_fmri
from fsl.wrappers import fslmaths
import glob
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects


In [2]:
# This will clear the view and remove our visualization totally:

def reset_overlays():
    """
    Clears view and completely remove visualization. All files opened in FSLeyes are closed.
    The view (along with any color map) is reset to the regular ortho panel.
    """
    l = frame.overlayList
    while(len(l)>0):
        del l[0]
    frame.removeViewPanel(frame.viewPanels[0])
    # Put back an ortho panel in our viz for future displays
    frame.addViewPanel(OrthoPanel)
    
def display_atlas_corresp(atlas_xml_path):
    """
    Given a path to an XML, extract all region labels and print their correspondance with numerical values
    of the nifti atlas file. (FSL-based convention)
    """
    root=ET.parse(atlas_xml_path).getroot()
    for type_tag in root.findall('data/label'):
        value = type_tag.get('index')
        print(int(value)+1, ':', type_tag.text)

The first part of this week is devoted to both installation and this notebook.
Before attempting this notebook, please make sure you've followed installation steps in installation.md
This week, you will get a bit more familiar with visualization tools to view MRI images, diffusion MR images and lastly atlas visualization. We will introduce the different tools on the fly, so that you get a good idea of how to use them for your projects!

Let's get started.


# 1. Visualization of MR images

These images are typically in a very peculiar format: the <a href="https://brainder.org/2012/09/23/the-nifti-file-format/">NIfTI file format</a>. You will notice that files have the .nii extension, or .nii.gz extension (which is nothing more than a compressed NIfTI file). While a full discourse on the format itself is a bit out of scope for this exercise, there are several things one should be aware of:
- A NIfTI file under the hood comprises two things, a header, and an image. The header keeps informations on the acquisition, such as dimensions of the voxels used, number of slices for each dimension, TR, affine transformation, sequence parameters...Some scanners (specifically SIEMENS scanners) will also store in the header information about the timing acquisitions of each slice, which is particularly handy when you need to perform slice-timing correction. The image itself is the data that has been acquired: the 3D volume in the case of a structural MR image!
- Because of its peculiar format and its 3D nature (for structurals again! a functional MRI would be 4D as it is a collection of 3D acquisitions), it is typically not possible to visualize them with usual image processing libraries such as ImageMagick: we need specialized software !

In this course, we will use <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki">FSL</a>.


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b><img src="https://fsl.fmrib.ox.ac.uk/fsl/wiki_static/fsl/img/fsl-logo-x2.png">FSL</b></p>
<p style='text-indent: 10px;'>
FSL (FMRIB Software Library) is a library of analysis and visualization used for MRI, fMRI and DTI. It is typically run through command line. To make it easier on you, we are relying on the python interface of FSL, but you can also install the complete version on your system (it's free!) by following the instructions <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation">here</a>.<br>
    FSL is not the only library available! <a href="https://afni.nimh.nih.gov/">AFNI</a> and <a href="https://www.fil.ion.ucl.ac.uk/spm/">SPM</a> are two such other libraries, to quote two of the most well known. They all offer different flexibilities and ways to program. As an example, SPM is mainly reliant on MATLAB, while AFNI is heavy on the command-line. To serve as an easy hand-on introduction, we are using FSL, because Python is a language with which most of you are likely to be familiar at some level. Note that should you wish to use the command line commands of FSL, you can get quite detailed into the steps you wish to apply.
</p></span>
</div>

If you did launch this notebook using FSLeyes (if not, have a look at the README again!), you will now witness the unlimited power of the interactive session. For now you should have something empty, ie it might look like this:
<img src="imgs/fsleyes_empty.png">
<br>
### First visualization

We will add a first image in FSLeyes. This is done by specifying the absolute path to the image (in .nii or .nii.gz format !) 

In our case, we will start by showing a standard anatomical image: the brain in MNI space.
This brain is located in the path $FSLDIR/data/standard/MNI512_T1_0.5mm.nii.gz
Knowing this path, we can use FSLeyes load() function to directly load the image in the software interactively! 
Run the cell below:

In [3]:
load(op.expandvars('$FSLDIR/data/standard/MNI152_T1_0.5mm'))

Image(MNI152_T1_0.5mm, /usr/local/fsl/data/standard/MNI152_T1_0.5mm.nii.gz)

You should see something that looks like this:
<img src="imgs/mni_template_ex.png">

First of all, you should observe the specific contrast used: this is a T1 contrast, typically used for anatomical scans. Notice here we have picked a rather exquisite resolution: 0.5mm (meaning here a given voxel in 3D is 0.5 x 0.5 x 0.5 mm³)! 


Every time you move in a direction, you move across what are called **slices**. 


Can you make out the different types of tissues in the brain, just at a glance? Try to pinpoint where the majority of grey matter is found, based on contrast only.


Using your mouse, don't hesitate to play around the display a bit! Notice the 3-view display, showing in order sagittal, coronal and axial views of the brain. You can disable each view individually, by clicking on the appropriate buttons:
<img src="imgs/fsl_view_buttons.png"><br>


On the **sagittal slice**, you can see two letters at each side of the brain: P and A, for Posterior and Anterior, respectively. Moving only along the sagittal axis, can you approximately guess where the eye sockets are? (Note: you are allowed to move only along the sagittal axis, but you can still look at the other views, especially the coronal view!)

### Atlases and brain masks 

FSL has many functionalities. When looking at brain data, one interesting aspect is the one of atlases: figuring out which region one is looking at is useful to answer neurological questions or even to perform sanity checks on your data. As an example, if your participants are involved in a clicking task but no activity is visible in motor areas, you're probably doing something wrong somewhere. But to detect this, you obviously need to be able to distinguish **where** motor areas are located! 
One of its most basic uses is the ability to overlay images on top of each other. To demonstrate this one, we will load an atlas on top of our brain. 

<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>Atlases in MRI</b></p>
<p style='text-indent: 10px;'>
There are many atlases available, depending on your interest; some provide insight into subcortical structures, whereas others will partition the cortex. Atlases can be functional or anatomical, data-driven or labeled by experts, bilateral... All these choices will induce differences in the outlined regions, obviously. You should also be aware of the fact that atlases will provide you an image at the **population level** but will not be perfectly accurate at the population level.


In any case, an atlas is typically represented also a NIfTI file. Each region in the atlas - a region of interest (ROI) - takes a **unique** value to distinguish from the others. A companion file usually indicates the correspondance between the value and the region.
    
<img src="imgs/atlas_example.png">
    <center><i>One example of an atlas. Notice the colors matching in each hemisphere: this atlas does not differentiate between the two hemispheres for its regions.</i></center>
</p></span>
</div>


#### Loading an atlas and visualizing its regions

We will load our T1 (in 1mm resolution this time) and overlay an atlas image on top.
The colormap of the atlas can be set, which allows us to better visualize the different regions. In the right inferior corner of FSLeyes, you can view the value of the voxel at which the green cross is centered.
<img src="imgs/cross_value_fsleyes.png">


In [4]:
reset_overlays()
load(op.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm'))

Image(MNI152_T1_1mm, /usr/local/fsl/data/standard/MNI152_T1_1mm.nii.gz)

In [5]:
# Loads the atlas image
load(op.expandvars('$FSLDIR/data/atlases/MNI/MNI-maxprob-thr25-1mm.nii.gz'))
# Because there are two images in the list of overlays, the 2nd image being the atlas,
# we can very easily target its colormap and change it like so:
displayCtx.getOpts(overlayList[1]).cmap = 'Render3'

The companion file can be found in the FSL directory of the atlases, in xml format.

Here is the correspondance of values to regions in this atlas: 

In [6]:
display_atlas_corresp(op.expandvars('$FSLDIR/data/atlases/MNI.xml'))
# if this is not shown here -- check on the terminal!

Using this equivalence, you can now locate and identify the different structures annotated by this atlas. Which sentence(s) is(are) accurate **for this atlas**?
- [ ] The thalamus is ventro-posterior of the caudate nucleus 
- [ ] The insula is a region anterior to the temporal lobe, lateral to the putamen and medial to the frontal cortex.
- [ ] The posterior precentral gyrus is part of the occipital lobe

#### Using atlases as masks

Now that you understand how atlases are handled, we can move on to another interest topic: masking data with an atlas. This can become handy for several reasons, the first being if you want to analyze data coming from a specific region. The mask can be created either in Python or through FSL. We will teach you both, so that you can choose whichever method you prefer. Let us create a mask of the frontal lobe, as an example.

##### Method 1: Python-based creation of the mask

The steps are simple:
1. Load the atlas as a regular file and extract the data as a numpy array
2. Create a binary mask of the frontal lobe
3. Write back to disk the mask with only the part of interest set to 1

Just to make sure you're still following and not falling asleep, we've provided you with a function that does the above steps...But you need to call it with the proper arguments to extract the frontal lobe *and* give a name to the file! By default, the mask will be named mymask.nii.

Lastly, to convince you that this mask is indeed the real deal, we will display it in FSLeyes!

In [7]:
def make_mask_from_atlas(mask_value, mask_name="mymask.nii"):
    # Load the atlas with nibabel
    atlas = nib.load(op.expandvars('$FSLDIR/data/atlases/MNI/MNI-maxprob-thr25-1mm.nii.gz'))
    # Extract the atlas data as a numpy array
    atlas_data = atlas.get_fdata()
    # The mask is set to true only in voxels which match the target mask_value
    mask_data = atlas_data == mask_value
    # Create a new nifti image with nibabel
    mask_img = nib.Nifti1Image(mask_data.astype(np.uint8), atlas.affine, atlas.header)
    # Save the image
    nib.save(mask_img, mask_name)  

In [9]:
mask_name='first_mask.nii' # Fill it with the name you want to give your mask!
mask_value=7 # Fill it with the value of the region you want to extract from the atlas!
make_mask_from_atlas(mask_value, mask_name)

load(mask_name)

Image(first_mask, /Users/simonlee/NX-421/Exercises 1-20220921/first_mask.nii)

##### Method 2: FSL-based creation of the mask

Again the steps are not too hard! You simply need to apply exactly the logic we have applied in method 1 to take the atlas, set all values of interest to 1 and the rest to zero. We will do so using a simple method: we will lower threshold the image, setting to zero anything **below** the provided threshold. Then, we will set anything **above** a second threshold to zero. Anything that is non zero by this point, we will set to 1.


We provide you with a simple function that accepts the two thresholds described above, named lower_threshold and upper_threshold. Decide on their value so as to extract again the *frontal lobe*. Likewise, the function will by default save the file as mymask.nii. Feel free to give it a more explicit name!

In [10]:
def make_mask_from_atlas_fsl(lower_threshold, upper_threshold, mask_name="mymask.nii"):
    fslmaths(op.expandvars('$FSLDIR/data/atlases/MNI/MNI-maxprob-thr25-1mm.nii.gz')).thr(lower_threshold).uthr(upper_threshold).bin().run(mask_name)

In [11]:
mask_name='commandlinemask.nii'
lower_threshold=1
upper_threshold=5
make_mask_from_atlas_fsl(lower_threshold, upper_threshold, mask_name)

load(glob.glob(mask_name + '*')[0])

Image(commandlinemask, /Users/simonlee/NX-421/Exercises 1-20220921/commandlinemask.nii.gz)

## 2. Viewing functional MR images

What you've learnt so far in visualization can very easily be applied to fMRI images. To showcase this, we will load one fMRI session. Based on what you have seen on MR images, and knowing that fMRI is a collection of MR images, what is true?

- [ ] An fMRI recording is in 4D because we add time
- [ ] An fMRI recording is a collection of T1-weighted images
- [ ] An fMRI recording is a collection of BOLD-weighted images
- [ ] An fMRI can be both T1 or T2 weighted, depending on the TR and TE


In [12]:
reset_overlays()
load(fetch_development_fmri(n_subjects=1, verbose=1, age_group='both')['func'][0])

Image(sub-pixar123_task-pixar_space-MNI152NLin2009cAsym_desc-preproc_bold, /Users/simonlee/nilearn_data/development_fmri/development_fmri/sub-pixar123_task-pixar_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz)

Please pay attention to the fMRI's voxel dimensions:

In [13]:
'Dimensions: ' + ' x '.join([str(i) + 'mm' for i in  nib.load(fetch_development_fmri(n_subjects=1, verbose=1, age_group='both')['func'][0]).header.get_zooms()])[:-2] + 's'

'Dimensions: 4.0mm x 4.0mm x 4.0mm x 1.0s'

Do they match what you would expect? Compared to the anatomical you have seen so far, why do you think we observe such a difference in resolution?

### fMRI as a movie

As you've no doubt noticed, FSLeyes has one little sneaky button: the movie button!
If you click on it, it will start to play the fMRI recording as a movie, allowing you to appreciate how the signal is fluctuating across time!


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>🐞 Troubleshooting: fMRI movie is flickering 🐞</b></p>
<p style='text-indent: 10px;'>
    
If you enable the movie with FSLeyes synchronized with Python, you might not see a very pleasant movie, but more something along the lines of dark flickers with some occasional brains appearing in between.
<img src="imgs/buggy_fmri_display.gif">
    
This happens because FSLeyes is attempting to synchronize with Python and - well, there is a sluggish response between the two. Alleviating this behaviour is very easy! Simply disable the <b>Synchronize movie updates</b> option:
<img src="imgs/settings_button.png">
    <center><i>Click on the settings</i></center>
<img src="imgs/settings_menu.png">
    <center><i>Make sure to uncheck the Synchronize movie updates option</i></center>

</p></span>
</div>



## 3. Viewing diffusion MR images

Now that you are a bit familiar with FSLeyes, you should know you can also view other modalities, besides anatomical MR images with FSL; you can view functional data, but also diffusion weighted MR images!
We will show you how to view the fibers for an examplar dataset available in the DIPY library - a library specialized in diffusion imaging.

DIPY offers other visualizations that might interest you, so feel free to have a look in their documentation: https://dipy.org/tutorials/

In [49]:
path_bundle = fetch_bundles_2_subjects()[1]

In [50]:
files = glob.glob(op.join(path_bundle, "bundles_2_subjects/subj_1/bundles/*.trk")) 
reset_overlays()
load(op.join(path_bundle, "bundles_2_subjects/subj_1/t1_warped.nii.gz"))
[load(f) for f in files]